In [ ]:
import os
import argparse
import re
import time
import httplib2
import pickle
from pathlib import Path

from apiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from dotenv import load_dotenv

load_dotenv()


# Data Collection

## Google Analytics data

### Initialise GA

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/analytics.readonly"]

# Path to client_secrets.json file.
CLIENT_SECRETS_PATH = "client_secrets.json"
VIEW_ID = os.environ["VIEW_ID"]  # client dependent

# Parse command-line arguments.
parser = argparse.ArgumentParser(
    formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser]
)
flags = parser.parse_args([])

# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    CLIENT_SECRETS_PATH,
    scope=SCOPES,
    message=tools.message_if_missing(CLIENT_SECRETS_PATH),
)

# Prepare credentials, and authorize HTTP object with them.
# If the credentials don't exist or are invalid run through the native client
# flow. The Storage object will ensure that if successful the good
# credentials will get written back to a file.
storage = file.Storage("analyticsreporting.dat")
credentials = storage.get()
if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
http = credentials.authorize(http=httplib2.Http())

# Build the service object.
analytics = build("analyticsreporting", "v4", http=http)

### Query GA API for GA Data

Not all metrics and dimension can be queried together in GA. In order to create only valid combinations of metrics and dimensions, four separate requests were made. 

#### Request 1

In [ ]:
response_1 = (
    analytics.reports()
    .batchGet(
        body={
            "reportRequests": [
                {
                    "viewId": VIEW_ID,
                    "dateRanges": [{"startDate": "181daysAgo", "endDate": "yesterday"}],
                    "samplingLevel": "LARGE",
                    "metrics": [
                        {"expression": "ga:sessions"},
                        {"expression": "ga:pageviews"},
                        {"expression": "ga:uniquePageviews"},
                        {"expression": "ga:entrances"},
                        {"expression": "ga:bounces"},
                        {"expression": "ga:exits"},
                        {"expression": "ga:timeOnPage"},
                        {"expression": "ga:sessionDuration"},
                        {"expression": "ga:pageviewsPerSession"},
                    ],
                    "dimensions": [{"name": "ga:pagePath"}],
                    "dimensionFilterClauses": [
                        {
                            "filters": [
                                {
                                    "dimensionName": "ga:hostname",
                                    "not": False,
                                    "operator": "PARTIAL",
                                    "expressions": os.environ["HOSTNAME"],  # client dependent
                                    "caseSensitive": False,
                                }
                            ],
                        }
                    ],
                    "metricFilterClauses": [{}],
                    "pageSize": 100000,
                }
            ]
        }
    )
    .execute()
)

#### Request 2

In [ ]:
response_2 = (
    analytics.reports()
    .batchGet(
        body={
            "reportRequests": [
                {
                    "viewId": VIEW_ID,
                    "dateRanges": [{"startDate": "181daysAgo", "endDate": "yesterday"}],
                    "samplingLevel": "LARGE",
                    "dimensions": [{"name": "ga:pagePath"}, {"name": "ga:segment"}],
                    "dimensionFilterClauses": [
                        {
                            "filters": [
                                {
                                    "dimensionName": "ga:hostname",
                                    "not": False,
                                    "operator": "PARTIAL",
                                    "expressions": os.environ["HOSTNAME"],  # client dependent
                                    "caseSensitive": False,
                                }
                            ],
                        }
                    ],
                    "metrics": [
                        {"expression": "ga:sessions"},
                        {"expression": "ga:pageviews"},
                        {"expression": "ga:uniquePageviews"},
                        {"expression": "ga:entrances"},
                        {"expression": "ga:bounces"},
                    ],
                    "metricFilterClauses": [{}],
                    # Organic Traffic - built in GA segment ID 5
                    "segments": [{"segmentId": "gaid::-5"},],
                    "pageSize": 100000,
                }
            ]
        }
    )
    .execute()
)

#### Request 3

In [ ]:
response_3 = (
    analytics.reports()
    .batchGet(
        body={
            "reportRequests": [
                {
                    "viewId": VIEW_ID,
                    "dateRanges": [{"startDate": "181daysAgo", "endDate": "yesterday"}],
                    "samplingLevel": "LARGE",
                    "dimensions": [{"name": "ga:pagePath"}],
                    "dimensionFilterClauses": [
                        {
                            "filters": [
                                {
                                    "dimensionName": "ga:hostname",
                                    "not": False,
                                    "operator": "PARTIAL",
                                    "expressions": os.environ["HOSTNAME"],  # client dependent
                                    "caseSensitive": False,
                                },
                            ],
                        },
                        {
                            "operator": "OR",
                            "filters": [
                                {
                                    "dimensionName": "ga:eventCategory",
                                    "not": False,
                                    "operator": "REGEXP",
                                    "expressions": ".*(email|mailto|contact|tel|phone|call|number|conversion|purchase|download|pdf|enquiry).*",
                                    "caseSensitive": False,
                                },
                                {
                                    "dimensionName": "ga:eventAction",
                                    "not": False,
                                    "operator": "REGEXP",
                                    "expressions": ".*(email|mailto|contact|tel|phone|call|number|conversion|purchase|download|pdf|enquiry).*",
                                    "caseSensitive": False,
                                },
                            ],
                        },
                    ],
                    "metrics": [{"expression": "ga:uniqueEvents"},],
                    "metricFilterClauses": [{}],
                    "pageSize": 100000,
                }
            ]
        }
    )
    .execute()
)

#### Request 4

In [ ]:
response_4 = (
    analytics.reports()
    .batchGet(
        body={
            "reportRequests": [
                {
                    "viewId": VIEW_ID,
                    "dateRanges": [{"startDate": "181daysAgo", "endDate": "yesterday"}],
                    "samplingLevel": "LARGE",
                    "dimensions": [{"name": "ga:goalCompletionLocation"},],
                    "dimensionFilterClauses": [
                        {
                            "filters": [
                                {
                                    "dimensionName": "ga:goalCompletionLocation",
                                    "not": False,
                                    "operator": "REGEXP",
                                    "expressions": f"^{os.environ['HOSTNAME']}",  # client dependent
                                    "caseSensitive": False,
                                },
                            ]
                        },
                    ],
                    "metrics": [
                        {"expression": "ga:goal5Completions"},  # client dependent
                        {"expression": "ga:goal7Completions"},  # client dependent
                        {"expression": "ga:goal9Completions"},  # client dependent
                    ],
                    "metricFilterClauses": [
                        {
                            "operator": "OR",
                            "filters": [
                                {
                                    "metricName": "ga:goal5Completions",  # client dependent
                                    "not": False,
                                    "operator": "GREATER_THAN",
                                    "comparisonValue": "0",
                                },
                                {
                                    "metricName": "ga:goal7Completions",  # client dependent
                                    "not": False,
                                    "operator": "GREATER_THAN",
                                    "comparisonValue": "0",
                                },
                                {
                                    "metricName": "ga:goal9Completions",  # client dependent
                                    "not": False,
                                    "operator": "GREATER_THAN",
                                    "comparisonValue": "0",
                                },
                            ],
                        }
                    ],
                    "pageSize": 100000,
                }
            ]
        }
    )
    .execute()
)

### Turn GA API response into Dataframe

#### Create DataFrame 1

In [ ]:
for report in response_1.get("reports", {}):
    rows = report.get("data", {}).get("rows", [])

    sessions = []
    pages = []
    pageviews = []
    unique_pageviews = []
    entrances = []
    bounces = []
    exits = []
    time_on_page = []
    session_duration = []
    pages_per_session = []

    for row in rows:
        dimensions = row.get("dimensions", [])
        metrics = row.get("metrics", [])

        pages.append(dimensions[0])

        sessions.append(metrics[0]["values"][0])
        pageviews.append(metrics[0]["values"][1])
        unique_pageviews.append(metrics[0]["values"][2])
        entrances.append(metrics[0]["values"][3])
        bounces.append(metrics[0]["values"][4])
        exits.append(metrics[0]["values"][5])
        time_on_page.append(metrics[0]["values"][6])
        session_duration.append(metrics[0]["values"][7])
        pages_per_session.append(metrics[0]["values"][8])

    ga_data_1 = pd.DataFrame(
        {
            "Page": pages,
            "Sessions": sessions,
            "Pageviews": pageviews,
            "Unique Pageviews": unique_pageviews,
            "Entrances": entrances,
            "Bounces": bounces,
            "Time on Page (s)": time_on_page,
            "Exits": exits,
            "Session Duration (s)": session_duration,
            "Pages / Session": pages_per_session,
        }
    )

#### Create DataFrame 2

In [ ]:
for report in response_2.get("reports", {}):
    rows = report.get("data", {}).get("rows", [])

    sessions = []
    pages = []
    pageviews = []
    unique_pageviews = []
    entrances = []
    bounces = []

    for row in rows:
        dimensions = row.get("dimensions", [])
        metrics = row.get("metrics", [])

        pages.append(dimensions[0])

        sessions.append(metrics[0]["values"][0])
        pageviews.append(metrics[0]["values"][1])
        unique_pageviews.append(metrics[0]["values"][2])
        entrances.append(metrics[0]["values"][3])
        bounces.append(metrics[0]["values"][4])

    ga_data_2 = pd.DataFrame(
        {
            "Page": pages,
            "Organic Pageviews": pageviews,
            "Organic Unique Pageviews": unique_pageviews,
            "Organic Entrances": entrances,
            "Organic Bounces": bounces,
        }
    )

#### Create DataFrame 3

In [ ]:
for report in response_3.get("reports", {}):
    rows = report.get("data", {}).get("rows", [])

    pages = []
    unique_events = []

    for row in rows:
        dimensions = row.get("dimensions", [])
        metrics = row.get("metrics", [])

        pages.append(dimensions[0])

        unique_events.append(metrics[0]["values"][0])

    ga_data_3 = pd.DataFrame({"Page": pages, "Unique Events": unique_events,})

#### Create DataFrame 4

In [ ]:
for report in response_4.get("reports", {}):
    rows = report.get("data", {}).get("rows", [])

    pages = []
    goal_completions_1 = []
    goal_completions_2 = []
    goal_completions_3 = []

    for row in rows:
        dimensions = row.get("dimensions", [])
        metrics = row.get("metrics", [])

        pages.append(dimensions[0])

        goal_completions_1.append(metrics[0]["values"][0])
        goal_completions_2.append(metrics[0]["values"][1])
        goal_completions_3.append(metrics[0]["values"][2])

    ga_data_4 = pd.DataFrame(
        {
            "Page": pages,
            "Goal Completions (1)": goal_completions_1,
            "Goal Completions (2)": goal_completions_2,
            "Goal Completions (3)": goal_completions_3,
        },
    )

### Merge GA Data

In [ ]:
ga_data_merged_1 = pd.merge(ga_data_1, ga_data_2, on="Page", how="outer")

ga_data_merged_2 = pd.merge(ga_data_3, ga_data_4, on="Page", how="outer")

ga_data_merged_final = pd.merge(
    ga_data_merged_1, ga_data_merged_2, on="Page", how="outer"
)

### Clean and Prepare GA Data

#### Set data types and fill N/A's

In [ ]:
ga_data_merged_final["Page"] = ga_data_merged_final["Page"].astype("string")

ga_data_merged_final["Sessions"] = (
    ga_data_merged_final["Sessions"].fillna(0).astype("int")
)
ga_data_merged_final["Pageviews"] = (
    ga_data_merged_final["Pageviews"].fillna(0).astype("int")
)
ga_data_merged_final["Unique Pageviews"] = (
    ga_data_merged_final["Unique Pageviews"].fillna(0).astype("int")
)
ga_data_merged_final["Entrances"] = (
    ga_data_merged_final["Entrances"].fillna(0).astype("int")
)
ga_data_merged_final["Bounces"] = (
    ga_data_merged_final["Bounces"].fillna(0).astype("int")
)
ga_data_merged_final["Exits"] = ga_data_merged_final["Exits"].fillna(0).astype("int")
ga_data_merged_final["Organic Pageviews"] = (
    ga_data_merged_final["Organic Pageviews"].fillna(0).astype("int")
)
ga_data_merged_final["Organic Unique Pageviews"] = (
    ga_data_merged_final["Organic Pageviews"].fillna(0).astype("int")
)
ga_data_merged_final["Organic Entrances"] = (
    ga_data_merged_final["Organic Entrances"].fillna(0).astype("int")
)
ga_data_merged_final["Organic Bounces"] = (
    ga_data_merged_final["Organic Bounces"].fillna(0).astype("int")
)
ga_data_merged_final["Unique Events"] = (
    ga_data_merged_final["Unique Events"].fillna(0).astype("int")
)
ga_data_merged_final["Goal Completions (1)"] = (
    ga_data_merged_final["Goal Completions (1)"].fillna(0).astype("int")
)
ga_data_merged_final["Goal Completions (2)"] = (
    ga_data_merged_final["Goal Completions (2)"].fillna(0).astype("int")
)
ga_data_merged_final["Goal Completions (3)"] = (
    ga_data_merged_final["Goal Completions (3)"].fillna(0).astype("int")
)

ga_data_merged_final["Time on Page (s)"] = (
    ga_data_merged_final["Time on Page (s)"].fillna(0).astype("float")
)
ga_data_merged_final["Session Duration (s)"] = (
    ga_data_merged_final["Session Duration (s)"].fillna(0).astype("float")
)
ga_data_merged_final["Pages / Session"] = (
    ga_data_merged_final["Pages / Session"].fillna(0).astype("float")
)

#### Clean page data

In [ ]:
from constants import COUNTRY_CODES, LANGUAGE_CODES

language_codes = "|".join(COUNTRY_CODES)
country_codes = "|".join(LANGUAGE_CODES)

ga_data_clean = ga_data_merged_final
ga_data_clean["Page"] = ga_data_clean["Page"].str.replace(
    ".googleweblight.com", "", regex=True
)  # client dependent
ga_data_clean["Page"] = ga_data_clean["Page"].apply(
    lambda x: x + "/" if x[-1] != "/" else x
)
ga_data_clean["Page"] = ga_data_clean["Page"].str.replace("\\?.*", "", regex=True)
ga_data_clean["Page"] = ga_data_clean["Page"].str.replace(
    f"{os.environ['HOSTNAME']}/({language_codes})(-({country_codes}))?/",
    f"{os.environ['HOSTNAME']}/",
    regex=True,
)

ga_data_clean = ga_data_clean[~ga_data_clean["Page"].str.contains("search")]

#### Aggregate data by Page

In [ ]:
ga_data_agg = (
    ga_data_clean.groupby("Page")
    .agg(
        {
            "Sessions": "sum",
            "Pageviews": "sum",
            "Unique Pageviews": "sum",
            "Entrances": "sum",
            "Bounces": "sum",
            "Exits": "sum",
            "Time on Page (s)": "sum",
            "Session Duration (s)": "sum",
            "Pages / Session": "max",
            "Organic Pageviews": "sum",
            "Organic Unique Pageviews": "sum",
            "Organic Entrances": "sum",
            "Organic Bounces": "sum",
            "Unique Events": "sum",
            "Goal Completions (1)": "sum",
            "Goal Completions (2)": "sum",
            "Goal Completions (3)": "sum",
        }
    )
    .sort_values(by="Pageviews")
)

#### Calculate ratio metrics

In [ ]:
ga_data_agg_final = ga_data_agg.copy()

ga_data_agg_final["Goal Completions (All)"] = (
    ga_data_merged_final["Goal Completions (1)"]
    + ga_data_merged_final["Goal Completions (2)"]
    + ga_data_merged_final["Goal Completions (3)"]
)

ga_data_agg_final["Entrance Rate"] = (
    ga_data_agg_final["Entrances"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Bounce Rate"] = (
    ga_data_agg_final["Bounces"] / ga_data_agg_final["Entrances"]
)
ga_data_agg_final["Avg. Time on Page (s)"] = (
    ga_data_agg_final["Time on Page (s)"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Exit Rate"] = (
    ga_data_agg_final["Exits"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Avg. Session Duration (s)"] = (
    ga_data_agg_final["Session Duration (s)"] / ga_data_agg_final["Sessions"]
)

ga_data_agg_final["Organic Entrance Rate"] = (
    ga_data_agg_final["Organic Entrances"] / ga_data_agg_final["Organic Pageviews"]
)
ga_data_agg_final["Organic Bounce Rate"] = (
    ga_data_agg_final["Organic Bounces"] / ga_data_agg_final["Organic Entrances"]
)

ga_data_agg_final["Event Rate"] = (
    ga_data_agg_final["Unique Events"] / ga_data_agg_final["Unique Pageviews"]
)

ga_data_agg_final["Conversion Rate (1)"] = (
    ga_data_agg_final["Goal Completions (1)"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Conversion Rate (2)"] = (
    ga_data_agg_final["Goal Completions (2)"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Conversion Rate (3)"] = (
    ga_data_agg_final["Goal Completions (3)"] / ga_data_agg_final["Pageviews"]
)
ga_data_agg_final["Conversion Rate (All)"] = (
    ga_data_agg_final["Goal Completions (All)"] / ga_data_agg_final["Unique Pageviews"]
)

ga_data_agg_final = pd.DataFrame(
    ga_data_agg_final[
        (ga_data_agg_final["Pageviews"] > 100) & (ga_data_agg_final["Sessions"] > 0)
    ]
)



ga_data_agg_final = ga_data_agg_final.fillna(0)

### Scrape Site for Meta Data

#### Derive page list from dataframe

In [ ]:
ga_data_agg_final["URL"] = "https://" + ga_data_agg_final.index.to_series()

url_list = ga_data_agg_final["URL"].to_list()

#### Make HTTP request and store meta data

In [ ]:
pickled_meta_data_file = Path("meta_data.pickle")
if pickled_meta_data_file.is_file():
    with open(pickled_meta_data_file, "rb") as f:
        pickled_meta_data = pickle.load(f)
else:
    pickled_meta_data = {}

meta_data_list = {}
for url in url_list:
    if url in pickled_meta_data.keys():
        continue
    else:
        response = requests.get(url)

        status_okay = [200, 204]
        if response.status_code in status_okay:
            html = BeautifulSoup(response.text, "html.parser")

            tree_selectors = [
                "page-body",
                "insightContent",
                "insightPrintShare",
                "authorProfile",
                "hide-print",
            ]  # client dependent location def
            meta_tags = [
                "title",
                "h1",
                "h2",
                "h3",
                "h4",
                "a",
                "ul",
                "ol",
                "p",
                "img",
                "video",
                "form",
                "iframe",
            ]
            meta_values = {}

            for tag in meta_tags:
                if tag == "title":
                    tag_values = []
                    tag_values += [tag.text for tag in html.find("head").find_all(tag)]
                    meta_values[tag] = tag_values
                elif tag != "a":
                    tag_values = []
                    for class_name in tree_selectors:
                        if html.find("div", class_=class_name):
                            tag_values += [
                                tag.text
                                for tag in html.find("div", class_=class_name).find_all(tag)
                            ]
                    meta_values[tag] = tag_values
                else:
                    tag_values = []
                    for class_name in tree_selectors:
                        if html.find("div", class_=class_name):
                            tag_values += [
                                tag["href"]
                                for tag in html.find("div", class_=class_name).find_all(
                                    tag, href=True
                                )
                            ]
                    meta_values[tag] = tag_values

            meta_data_list[url] = meta_values
        else:
            meta_data_list[url] = {}

        if len(meta_data_list.keys()) % 10 == 0:
            pickled_meta_data = {
                **pickled_meta_data,
                **meta_data_list
            }
            with open(pickled_meta_data_file, "wb") as f:
                pickle.dump(pickled_meta_data, f)

In [ ]:
meta_data_list = {
    **pickled_meta_data,
    **meta_data_list
}

#### Count occurrences of HTML elements and store in dataframe columns

In [ ]:
meta_data = pd.DataFrame()

for url in url_list:
    for tag, values in meta_data_list[url].items():
        meta_data.loc[url, f"# of {tag}"] = len(values)

#### Count characters of HTML elements and store in dataframe columns

In [ ]:
for url in url_list_sample:
    for tag, values in meta_data_list[url].items():
        if tag in ["title", "h1", "h2", "h3", "h4", "p"]:
            chars = 0
            count = 0
            for item in values:
                chars += len(item)
                count += 1
            if count > 0:
                if tag == "p":
                    meta_data.loc[url, f"Total # of characters {tag}"] = chars
                else:
                    meta_data.loc[url, f"Avg. # of characters {tag}"] = chars / count

#### Split out link types

In [ ]:
for url in url_list_sample:
    int_outbound_count = 0
    tel_count = 0
    email_count = 0
    ext_outbound_count = 0

    for tag, values in meta_data_list[url].items():

        if tag == "a":

            for item in values:
                if item and item[0] and item[0] == "/":
                    item = f"https://{os.environ['HOSTNAME']}" + item

                if not "?" in item and not "#" in item:
                    if {os.environ['HOSTNAME']} in item:
                        int_outbound_count += 1
                    elif "tel:" in item or "fax" in item:
                        tel_count += 1
                    elif "@" in item:
                        email_count += 1
                    else:
                        ext_outbound_count += 1

    meta_data.loc[url, "No of int outbound links"] = int_outbound_count
    meta_data.loc[url, "No of ext outbound links"] = ext_outbound_count
    meta_data.loc[url, "No of email links"] = email_count
    meta_data.loc[url, "No of tel links"] = tel_count

#### Calculate inbound links

In [ ]:
url_count = {url: 0 for url in url_list_sample}
meta_data["No of inbound links"] = 0

for url in url_list_sample:

    for tag, values in meta_data_list[url].items():

        if tag == "a":

            for item in values:

                if item and item[0] and item[0] == "/":
                    item = f"https://{os.environ['HOSTNAME']}" + item

                item = re.sub(
                    r"{}\/({})(\-({}))?/".format(
                        {os.environ['HOSTNAME']}, language_codes, country_codes
                    ),
                    f"{os.environ['HOSTNAME']}/",
                    item,
                )

                if item and item[-1] and item[-1] != "/":
                    item = item + "/"

                if item in url_count:
                    url_count[item] += 1

                    meta_data.loc[url, "No of inbound links"] = url_count[item]

#### Calculate page path level

In [ ]:
page_path_levels = {}

for url in url_list_sample:
    page_path = len(url.split("/")) - 4
    page_path_levels[url] = page_path
    meta_data.loc[url, "Page path level"] = page_path_levels[url]

meta_data = meta_data.fillna(0)

### Merge GA and meta data

In [ ]:
meta_data["URL"] = meta_data.index

all_data = pd.merge(
    meta_data, ga_data_agg_final_sample, on="URL", how="left"
).set_index("URL")

# Statistical Analysis

## Google Analytics Data

### Descriptive statistics

In [ ]:
descr_stats_ga = pd.DataFrame(
    {
        "Sessions": all_data["Sessions"].describe(),
        "Pageviews": all_data["Pageviews"].describe(),
        "Organic Pageviews": all_data["Organic Pageviews"].describe(),
        "Unique Pageviews": all_data["Unique Pageviews"].describe(),
        "Organic Entrances": all_data["Organic Entrances"].describe(),
        "Entrances": all_data["Entrances"].describe(),
        "Entrance Rate": all_data["Entrance Rate"].describe(),
        "Organic Entrance Rate": all_data["Organic Entrance Rate"].describe(),
        "Bounces": all_data["Bounces"].describe(),
        "Organic Bounces": all_data["Organic Bounces"].describe(),
        "Bounce Rate": all_data["Bounce Rate"].describe(),
        "Organic Bounce Rate": all_data["Organic Bounce Rate"].describe(),
        "Exits": all_data["Exits"].describe(),
        "Exit Rate": all_data["Exit Rate"].describe(),
        "Avg. Session Duration (s)": all_data["Avg. Session Duration (s)"].describe(),
        "Avg. Time on Page (s)": all_data["Avg. Time on Page (s)"].describe(),
        "Pages / Session": all_data["Pages / Session"].describe(),
        "Unique Events": all_data["Unique Events"].describe(),
        "Event Rate": all_data["Event Rate"].describe(),
        "Goal Completions (All)": all_data["Goal Completions (All)"].describe(),
        "Conversion Rate (1)": all_data["Conversion Rate (1)"].describe(),
        "Conversion Rate (2)": all_data["Conversion Rate (2)"].describe(),
        "Conversion Rate (3)": all_data["Conversion Rate (3)"].describe(),
        "Conversion Rate (All)": all_data["Conversion Rate (All)"].describe(),
    }
)

descr_stats_ga

### Distribution

In [ ]:
def plot_distribution_score_metrics(df, skew_table, column_list, transf_type):
    fig, axes = plt.subplots(8, 3, figsize=(30, 30))
    axes_list = [ax for sublist in axes for ax in sublist]
    plt.subplots_adjust(wspace=0.5, hspace=0.5)

    for i, column in enumerate(column_list):
        sns.histplot(df, x=column, bins=20, kde=True, ax=axes_list[i]).set_title(
            f"{transf_type} skew: {skew_table[column]}"
        )
    plt.show()

In [ ]:
columns_list_score_metrics = [
    "Sessions",
    "Entrances",
    "Pageviews",
    "Organic Pageviews",
    "Unique Pageviews",
    "Organic Entrances",
    "Entrance Rate",
    "Organic Entrance Rate",
    "Bounces",
    "Organic Bounces",
    "Bounce Rate",
    "Organic Bounce Rate",
    "Exits",
    "Exit Rate",
    "Avg. Session Duration (s)",
    "Avg. Time on Page (s)",
    "Pages / Session",
    "Unique Events",
    "Event Rate",
    "Conversion Rate (1)",
    "Conversion Rate (2)",
    "Conversion Rate (3)",
    "Conversion Rate (All)",
    "Goal Completions (All)",
]

#### Without transformation

In [ ]:
original_skew = {}
for column in columns_list_score_metrics:
    original_skew[column] = all_data[column].skew()

%time plot_distribution_score_metrics(all_data, original_skew, columns_list_score_metrics, "original")

#### After logarithmic transformation

In [ ]:
all_data_log_transf = pd.DataFrame()
log_skew = {}
for column in columns_list_score_metrics:
    all_data_log_transf[column] = np.log10(all_data[column].replace(0, 0.01))
    log_skew[column] = all_data_log_transf[column].skew()

%time plot_distribution_score_metrics(all_data_log_transf, log_skew, columns_list_score_metrics, "logarithmic")

#### After Box Cox transformation

In [ ]:
all_data_boxcox_transf = pd.DataFrame(index=all_data.index)
boxcox_skew = {}
for column in columns_list_score_metrics:
    try:
        all_data_boxcox_transf[column] = stats.boxcox(
            all_data[column].replace(0, 0.01)
        )[0]
        boxcox_skew[column] = all_data_boxcox_transf[column].skew()
    except:
        all_data_boxcox_transf[column] = all_data[column]
        boxcox_skew[column] = np.nan

%time plot_distribution_score_metrics(all_data_boxcox_transf, boxcox_skew, columns_list_score_metrics, "boxcox")

#### Comparing skews - without and with transformation

In [ ]:
score_metrics_skew_df = pd.DataFrame(
    data={
        "Original": original_skew.values(),
        "Logarithmic": log_skew.values(),
        "Boxcox": boxcox_skew.values(),
    },
    index=original_skew.keys(),
)
score_metrics_skew_df

## Meta data

### Descriptive statistics

In [ ]:
descr_stats_meta = pd.DataFrame(
    {
        "No of title": all_data["No of title"].describe(),
        "No of h1": all_data["No of h1"].describe(),
        "No of h2": all_data["No of h2"].describe(),
        "No of h3": all_data["No of h3"].describe(),
        "No of h4": all_data["No of h4"].describe(),
        "No of a": all_data["No of a"].describe(),
        "No of ul": all_data["No of ul"].describe(),
        "No of ol": all_data["No of ol"].describe(),
        "No of p": all_data["No of p"].describe(),
        "No of img": all_data["No of img"].describe(),
        "No of video": all_data["No of video"].describe(),
        "No of form": all_data["No of form"].describe(),
        "No of iframe": all_data["No of iframe"].describe(),
        "Avg. # of characters title": all_data["Avg. # of characters title"].describe(),
        "Avg. # of characters h1": all_data["Avg. # of characters h1"].describe(),
        "Avg. # of characters h2": all_data["Avg. # of characters h2"].describe(),
        "Avg. # of characters h3": all_data["Avg. # of characters h3"].describe(),
        "Avg. # of characters h4": all_data["Avg. # of characters h4"].describe(),
        "Total # of characters p": all_data["Total # of characters p"].describe(),
        "No of int outbound links": all_data["No of int outbound links"].describe(),
        "No of ext outbound links": all_data["No of ext outbound links"].describe(),
        "No of email links": all_data["No of email links"].describe(),
        "No of tel links": all_data["No of tel links"].describe(),
        "No of inbound links": all_data["No of inbound links"].describe(),
        "Page path level": all_data["Page path level"].describe(),
    }
)

descr_stats_meta

### Distribution

In [ ]:
def plot_distribution_meta(df, skew_table, column_list, transf_type):
    fig2, axes2 = plt.subplots(12, 2, figsize=(30, 50))
    axes_list2 = [ax for sublist in axes2 for ax in sublist]
    plt.subplots_adjust(wspace=0.5, hspace=0.5)

    for i, column in enumerate(column_list):
        sns.histplot(df, x=column, bins=20, kde=True, ax=axes_list2[i]).set_title(
            f"{transf_type} skew: {skew_table[column]}"
        )
    plt.show()

In [ ]:
columns_list_meta = [
    "No of title",
    "Avg. # of characters title",
    "No of h1",
    "Avg. # of characters h1",
    "No of h2",
    "Avg. # of characters h2",
    "No of h3",
    "Avg. # of characters h3",
    "No of h4",
    "Avg. # of characters h4",
    "No of p",
    "Total # of characters p",
    "No of int outbound links",
    "No of ext outbound links",
    "No of email links",
    "No of tel links",
    "No of inbound links",
    "No of form",
    "No of ul",
    "No of ol",
    "No of img",
    "No of video",
    "No of iframe",
    "Page path level",
]

#### Without transformation

In [ ]:
meta_original_skew = {}
for column in columns_list_meta:
    meta_original_skew[column] = all_data[column].skew()

%time plot_distribution_meta(all_data, meta_original_skew, columns_list_meta, "original")

#### After logarithmic transformation

In [ ]:
all_data_log_transf_meta = pd.DataFrame(index=all_data.index)
meta_log_skew = {}
for column in columns_list_meta:
    try:
        all_data_log_transf_meta[column] = np.log10(all_data[column].replace(0, 0.01))
        meta_log_skew[column] = all_data_log_transf_meta[column].skew()
    except:
        all_data_log_transf_meta[column] = all_data[column]
        meta_log_skew[column] = np.nan

%time plot_distribution_meta(all_data_log_transf_meta, meta_log_skew, columns_list_meta, "logarithmic")

#### After Box Cox transformation

In [ ]:
all_data_boxcox_transf_meta = pd.DataFrame(index=all_data.index)
meta_boxcox_skew = {}
for column in columns_list_meta:
    try:
        all_data_boxcox_transf_meta[column] = stats.boxcox(
            all_data[column].replace(0, 0.01)
        )[0]
        meta_boxcox_skew[column] = all_data_boxcox_transf_meta[column].skew()
    except:
        all_data_boxcox_transf_meta[column] = all_data[column]
        meta_boxcox_skew[column] = np.nan

%time plot_distribution_meta(all_data_boxcox_transf_meta, meta_boxcox_skew, columns_list_meta, "boxcox")

#### Comparing skews - without and with transformation

In [ ]:
meta_skew_df = pd.DataFrame(
    data={
        "Original": meta_original_skew.values(),
        "Logarithmic": meta_log_skew.values(),
        "Boxcox": meta_boxcox_skew.values(),
    },
    index=meta_original_skew.keys(),
)
meta_skew_df

## Correlations

### Feature Scaling

### Standardization (z-score scaling) - only use for (near) normally distributed data!

In [ ]:
def standardize_df(df):
    # Returns a copy of df with centred values
    cdf = df.copy()
    for column in cdf.columns:
        cdf[column] = (cdf[column] - cdf[column].mean()) / cdf[column].std()
    return cdf

In [ ]:
all_data_standard = standardize_df(all_data)

In [ ]:
# all_data_standard.describe()

### Normalization (Min-max scaling) - use for non-normally (skewed) distributed data

In [ ]:
def normalize_df(df):
    # Returns a copy of df with centred values
    cdf = df.copy()
    for column in cdf.columns:
        cdf[column] = (cdf[column] - cdf[column].min()) / (
            cdf[column].max() - cdf[column].min()
        )
    return cdf

In [ ]:
all_data_normal = normalize_df(all_data)

### Correlation matrix with statistical significance

In [ ]:
def create_correlation_matrix(df):
    rho = df.corr()
    p_values = df.corr(method=lambda x, y: stats.pearsonr(x, y)[1]) - np.eye(*rho.shape)
    significance_level = p_values.applymap(
        lambda x: "".join(["*" for alpha in [0.01, 0.05, 0.1] if x <= alpha])
    )
    return rho.round(2).astype(str) + " " + significance_level

In [ ]:
corr_matrix = create_correlation_matrix(all_data_normal).iloc[
    : meta_data.shape[1] - 1, meta_data.shape[1] - 1 :
]

In [ ]:
# corr_matrix

In [ ]:
def correlation_matrix_to_table(corr_matrix):
    feature_1_list = []
    feature_2_list = []
    value_list = []

    for column in corr_matrix.columns:
        for index in corr_matrix.index:
            value_list.append(corr_matrix.loc[index, column])
            feature_1_list.append(index)
            feature_2_list.append(column)

    return pd.DataFrame(
        {
            "Correlation": value_list,
            "Feature 1": feature_1_list,
            "Feature 2": feature_2_list,
        }
    )

In [ ]:
corr = correlation_matrix_to_table(corr_matrix)

In [ ]:
corr[["Corr. Strength", "Significance Level"]] = corr.Correlation.str.split(
    " ", expand=True,
)
corr["Corr. Strength"] = corr["Corr. Strength"].astype("float")
corr["Corr. Strength Absol."] = corr["Corr. Strength"].apply(lambda x: abs(x))

In [ ]:
corr.sort_values(by=["Corr. Strength Absol."], ascending=False)
corr_top = (
    corr[corr["Significance Level"].str.contains("\*")]
    .drop(labels="Correlation", axis=1)
    .sort_values(by=["Corr. Strength Absol."], ascending=False)
    .reset_index(drop=True)
)

In [ ]:
corr_top

# Scoring

In [ ]:
def plot_distribution_scores(df, rows, columns):
    fig, axes = plt.subplots(rows, columns, figsize=(30, 30))
    axes_list = [ax for sublist in axes for ax in sublist]
    plt.subplots_adjust(wspace=0.5, hspace=0.5)

    for i, column in enumerate(df.columns):
        sns.histplot(df, x=column, binwidth=1, kde=True, ax=axes_list[i])

    plt.show()

### Calculate scores

#### Catch scores

In [ ]:
scores_data = pd.DataFrame(index=all_data_boxcox_transf.index)

catch_metrics = [
    "Entrances",
    "Entrance Rate",
    "Bounce Rate",
    "Organic Entrances",
    "Organic Entrance Rate",
    "Organic Bounce Rate",
]

for i, metric in enumerate(catch_metrics):

    if "Bounce" in metric:
        scores_data[f"Catch Score {i+1} ({metric})"] = pd.cut(
            all_data_boxcox_transf[metric], 10, labels=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
        ).astype("int")

    else:
        scores_data[f"Catch Score {i+1} ({metric})"] = pd.cut(
            all_data_boxcox_transf[metric], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        ).astype("int")

scores_data["Catch Total Score"] = (
    scores_data.iloc[:, 0]
    + scores_data.iloc[:, 1]
    + scores_data.iloc[:, 2]
    + scores_data.iloc[:, 3]
    + scores_data.iloc[:, 4]
    + scores_data.iloc[:, 5]
) / 6

scores_data["Catch Total Score (Final)"] = scores_data["Catch Total Score"].round(1)

#### Connect scores

In [ ]:
connect_metrics = [
    "Avg. Time on Page (s)",
    "Exit Rate",
    "Unique Pageviews",
]

# Left out Avg. Session Duration, Pages / Session as this is sparse data due to it only being provided
# for sessions where the page was the landing page (penalising pages that aren't good landing pages)

# How to penalise unique pageviews score when page path level is low?

all_data_boxcox_transf.columns

for i, metric in enumerate(connect_metrics):
    if "Exit" in metric:
        scores_data[f"Connect Score {i+1} ({metric})"] = pd.cut(
            all_data_boxcox_transf[metric], 10, labels=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
        ).astype("int")

    else:
        scores_data[f"Connect Score {i+1} ({metric})"] = pd.cut(
            all_data_boxcox_transf[metric], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        ).astype("int")

scores_data["Connect Total Score"] = (
    scores_data.iloc[:, 8] + scores_data.iloc[:, 9] + scores_data.iloc[:, 10]
) / 3

scores_data["Connect Total Score (Final)"] = scores_data["Connect Total Score"].round(1)

#### Convert scores

In [ ]:
convert_metrics = [
    "Event Rate",
    "Unique Events",
    "Conversion Rate (1)",
    "Conversion Rate (2)",
    "Conversion Rate (3)",
    "Conversion Rate (All)",
    "Goal Completions (All)",
]

for i, metric in enumerate(convert_metrics):
    scores_data[f"Convert Score {i+1} ({metric})"] = pd.cut(
        all_data_boxcox_transf[metric], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    ).astype("int")

scores_data["Convert Total Score"] = (
    scores_data.iloc[:, 13]
    + scores_data.iloc[:, 14]
    + scores_data.iloc[:, 15]
    + scores_data.iloc[:, 16]
    + scores_data.iloc[:, 17]
    + scores_data.iloc[:, 18]
    + scores_data.iloc[:, 19]
) / 7

scores_data["Convert Total Score (Final)"] = scores_data["Convert Total Score"].round(1)

### Plot scores

#### Catch scores

In [ ]:
plot_distribution_scores(scores_data.iloc[:, :8], 4, 2)

#### Connect scores

In [ ]:
plot_distribution_scores(scores_data.iloc[:, 8:13], 3, 2)

#### Convert scores

In [ ]:
plot_distribution_scores(scores_data.iloc[:, 13:], 5, 2)

### Select top and bottom scoring URLs

In [ ]:
top_20_index = list(range(1, 21))
catch_top = (
    scores_data.sort_values(
        by=["Catch Total Score", "Catch Score 1 (Entrances)"], ascending=False
    )
    .iloc[0:20, 7]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)
connect_top = (
    scores_data.sort_values(
        by=["Connect Total Score", "Connect Score 3 (Unique Pageviews)"],
        ascending=False,
    )
    .iloc[0:20, 12]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)
convert_top = (
    scores_data.sort_values(
        by=["Convert Total Score", "Convert Score 7 (Goal Completions (All))"],
        ascending=False,
    )
    .iloc[0:20, -1]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)

In [ ]:
catch_bottom = (
    scores_data.sort_values(by=["Catch Total Score", "Catch Score 1 (Entrances)"])
    .iloc[0:20, 7]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)
connect_bottom = (
    scores_data.sort_values(
        by=["Connect Total Score", "Connect Score 3 (Unique Pageviews)"]
    )
    .iloc[0:20, 12]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)
convert_bottom = (
    scores_data.sort_values(
        by=["Convert Total Score", "Convert Score 7 (Goal Completions (All))"]
    )
    .iloc[0:20, -1]
    .reset_index()
    .set_index(pd.Series(top_20_index))
)

#### Catch scores

In [ ]:
catch_top

In [ ]:
catch_bottom

#### Connect scores

In [ ]:
connect_top

In [ ]:
connect_bottom

#### Convert scores

In [ ]:
convert_top

In [ ]:
convert_bottom

# Recommendations

## Meta data reference stats for top scoring pages

### Catch

In [ ]:
top_catch_meta = all_data.loc[convert_bottom["URL"],:].iloc[:,:meta_data.shape[1]-1]
top_catch_meta.mean()
top_catch_meta_df = pd.DataFrame({"Mean": top_catch_meta.mean(), "Max": top_catch_meta.max(), "Min": top_catch_meta.min()})

In [ ]:
top_catch_meta_df

### Connect

In [ ]:
top_connect_meta = all_data.loc[connect_bottom["URL"],:].iloc[:,:meta_data.shape[1]-1]
top_connect_meta.mean()
top_connect_meta_df = pd.DataFrame({"Mean": top_connect_meta.mean(), "Max": top_connect_meta.max(), "Min": top_connect_meta.min()})

In [ ]:
top_connect_meta_df

### Convert

In [ ]:
top_convert_meta = all_data.loc[convert_bottom["URL"],:].iloc[:,:meta_data.shape[1]-1]
top_convert_meta_df = pd.DataFrame({"Mean": top_convert_meta.mean(), "Max": top_convert_meta.max(), "Min": top_convert_meta.min()})

In [ ]:
top_convert_meta_df

## Most influencing factors

In [ ]:
def get_increase_action(row):
    if row["Corr. Strength"] > 0:
        increase_action = "by INCREASING"
    else:
        increase_action = "by DECREASING"
        
    return increase_action

def get_action_effect(row):
    if abs(row["Corr. Strength"]) > 0.7:
        action_effect = "may be INCREASED GREATLY"
    elif abs(row["Corr. Strength"]) > 0.5:
        action_effect = "may be INCREASED"
    else:
        action_effect = "may be INCREASED SLIGHTLY"
        
    return action_effect

def reverse_actions_and_effects(row):
    if row["Feature 2"] in ["Bounce Rate", "Organic Bounce Rate"]:
        if "INCREASING" in row["Action"]:
            row["Action"] = row["Action"].replace("INCREASING", "DECREASING")
        elif "DECREASING" in row["Action"]:
            row["Action"] = row["Action"].replace("DECREASING", "INCREASING")
            
        row["Effect"] = row["Effect"].replace("INCREASED", "DECREASED")
        
    return row

### Catch scores

In [ ]:
pd.set_option("display.max_colwidth",None)

corr_top_catch = corr_top[corr_top["Feature 2"].isin(catch_metrics)][["Feature 2","Feature 1","Corr. Strength"]]
corr_top_catch["Action"] = corr_top_catch.apply(get_increase_action, axis=1)
corr_top_catch["Effect"] = corr_top_catch.apply(get_action_effect, axis=1)
corr_top_catch.sort_values(by=["Feature 2"]).drop(columns=["Corr. Strength"])
corr_top_catch = corr_top_catch.groupby(["Feature 2", "Action", "Effect"]).agg({'Feature 1': lambda x: ', '.join(x)}).reset_index()
corr_top_catch = corr_top_catch[["Feature 2", "Effect", "Action", "Feature 1"]].apply(reverse_actions_and_effects, axis=1)
corr_top_catch

### Connect scores

In [ ]:
pd.set_option("display.max_colwidth",None)

corr_top_connect = corr_top[corr_top["Feature 2"].isin(connect_metrics)][["Feature 2","Feature 1","Corr. Strength"]]
corr_top_connect["Action"] = corr_top_connect.apply(get_increase_action, axis=1)
corr_top_connect["Effect"] = corr_top_connect.apply(get_action_effect, axis=1)
corr_top_connect.sort_values(by=["Feature 2"]).drop(columns=["Corr. Strength"])
corr_top_connect = corr_top_connect.groupby(["Feature 2", "Action", "Effect"]).agg({'Feature 1': lambda x: ', '.join(x)}).reset_index()
corr_top_connect = corr_top_connect[["Feature 2", "Effect", "Action", "Feature 1"]].apply(reverse_actions_and_effects, axis=1)
corr_top_connect

### Convert scores

In [ ]:
pd.set_option("display.max_colwidth",None)

corr_top_convert = corr_top[corr_top["Feature 2"].isin(convert_metrics)][["Feature 2","Feature 1","Corr. Strength"]]
corr_top_convert["Action"] = corr_top_convert.apply(get_increase_action, axis=1)
corr_top_convert["Effect"] = corr_top_convert.apply(get_action_effect, axis=1)
corr_top_convert.sort_values(by=["Feature 2"]).drop(columns=["Corr. Strength"])
corr_top_convert = corr_top_convert.groupby(["Feature 2", "Action", "Effect"]).agg({'Feature 1': lambda x: ', '.join(x)}).reset_index()
corr_top_convert = corr_top_convert[["Feature 2", "Effect", "Action", "Feature 1"]].apply(reverse_actions_and_effects, axis=1)
corr_top_convert

# Exporting Data into Google Sheets

In [ ]:
import gspread

In [ ]:
output_dfs = {
    "All scores": scores_data,
    "Top scoring pages 'catch'": catch_top,
    "Top scoring pages 'connect'": connect_top,
    "Top scoring pages 'convert'": convert_top,
    "Bottom scoring pages 'catch'": convert_bottom,
    "Bottom scoring pages 'connect'": convert_bottom,
    "Bottom scoring pages 'convert'": convert_bottom,  
    "Meta stats top scoring pages 'catch'": top_catch_meta_df,
    "Meta stats top scoring pages 'connect'": top_connect_meta_df,
    "Meta stats top scoring pages 'convert'": top_convert_meta_df,
    "Most influential factors 'catch'": corr_top_catch,
    "Most influential factors 'connect'": corr_top_connect,
    "Most influential factors 'convert'": corr_top_convert,
    "Correlations": corr_top
}

In [ ]:
client_name = os.environ['COMPANY_NAME']
title = "Content Performance Indicator Output"+" "+client_name
gc = gspread.oauth()

try:
    sh = gc.open(title)
except gspread.SpreadsheetNotFound:
    print("file does not exist yet")
    sh = gc.create(title)

sh_id = sh.id

In [ ]:
def create_worksheet_gsheet(dfs):
    for title, df in dfs.items():
        
        dfc = df.copy().reset_index()
        retry = True
        while retry:
            retry = False
            try:
                sh_ws = sh.worksheet(title)
                sh.del_worksheet(sh_ws)
                sh_ws = sh.add_worksheet(title, dfc.shape[0], dfc.shape[1])
            except gspread.WorksheetNotFound as e:
                print(e)
                try:
                    sh_ws = sh.add_worksheet(title, dfc.shape[0], dfc.shape[1])
                except gspread.exceptions.APIError as e:
                    print(e)
                    if e.response.status_code == 429:
                        print("1 - Wait 60s")
                        time.sleep(60)
                        retry = True
            except gspread.exceptions.APIError as e:
                print(e)
                if e.response.status_code == 429:
                    print("2 - Wait 60s")
                    time.sleep(60)
                    retry = True
            try:
                sh_ws.insert_rows(dfc.values.tolist())
                sh_ws.insert_row(dfc.columns.tolist())
                                
            except gspread.exceptions.APIError as e:
                print(e)
                if e.response.status_code == 429:
                    print("3 - Wait 60s")
                    time.sleep(60)
                    retry = True
                    
create_worksheet_gsheet(output_dfs)         